# Config

In [ ]:
%run <Fundraising_Config>

In [ ]:
# Resources
bronze_lakehouse_name = "{DYNAMICS_LAKEHOUSE_NAME}"

# Configuration
source_name = "Dynamics365"
source_id = "8d0a7c33-2a55-468b-9f12-1a14a43d8a19"

## Base classes

Foundation classes for Dynamics 365 synchronization operations.

In [ ]:
from pyspark.sql import DataFrame, functions as F
from pyspark.sql.functions import col, expr, lit, lower, trim, substring, current_timestamp
from pyspark.sql.types import StringType
from typing import Optional, Callable, Tuple
from abc import ABC
import logging


class Dynamics365BaseSync(ABC):
    """
    Abstract base class for Dynamics 365 synchronization operations.
    
    Provides common initialization for all Dynamics 365 sync classes.
    """
    
    def __init__(self, source_id: str, source_lakehouse: str, target_lakehouse: str):
        """
        Initialize the Dynamics 365 synchronizer.
        
        Args:
            source_id: SourceId GUID for this data source
            source_lakehouse: Bronze lakehouse name
            target_lakehouse: Silver lakehouse name
        """
        self.source_id = source_id
        self.source_lakehouse = source_lakehouse
        self.target_lakehouse = target_lakehouse
        self.mapping_table = f"{target_lakehouse}.SourceSystemIdMapping"

## Dynamics 365 option set synchronization

Synchronizes Dynamics 365 option sets to Silver dimension tables.
Performs full compare-and-sync: inserts new, updates changed, deletes removed options.

In [ ]:
from typing import Optional, Callable, Tuple
from pyspark.sql import DataFrame
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lit, lower, trim, substring, current_timestamp, expr
from pyspark.sql.types import StringType
import logging

class Dynamics365OptionsetSync:
    def __init__(
        self,
        *,
        source_id: str,
        source_lakehouse: str,
        target_lakehouse: str,
        mapping_table: Optional[str] = None,
    ) -> None:
        self.source_id = source_id
        self.source_lakehouse = source_lakehouse
        self.target_lakehouse = target_lakehouse
        # use provided mapping table or default
        self.mapping_table = mapping_table or f"{target_lakehouse}.SourceSystemIdMapping"

    def sync_optionset(
        self,
        entity_name: str,
        optionset_name: str,
        target_table: str,
        target_primary_key: str,
        is_global: bool = False,
        transform_func: Optional[Callable[[DataFrame], DataFrame]] = None,
    ) -> None:
        logging.info(f"🔄 Syncing optionset: {entity_name}.{optionset_name} → {target_table}")

        # 1) Bronze snapshot → canonical (Name, SourceSystemId, SourceId, Option for transforms)
        bronze_opts = self._read_bronze_options(entity_name, optionset_name, is_global)
        if transform_func is not None:
            logging.info("🔧 Applying custom transformation to optionset labels")
            bronze_opts = transform_func(bronze_opts)

        bronze_opts = (
            bronze_opts.select("Name", "SourceSystemId")
                       .dropna(subset=["Name", "SourceSystemId"])
                       .dropDuplicates(["SourceSystemId"])
                       .withColumn("SourceId", F.lit(self.source_id))
        )

        count_b = bronze_opts.count()
        logging.info(f"📊 Bronze options count: {count_b}")
        if count_b == 0:
            logging.warning("⚠️ No options found — skipping")
            return

        # 2) UPDATE (creator-owned only)
        upd_cnt = self._update_changed_options(bronze_opts, target_table, target_primary_key)

        # 3) INSERT (name-share discovery → mapping; then insert truly-new)
        ins_cnt = self._insert_new_options(bronze_opts, target_table, target_primary_key)

        logging.info(
            f"✅ Optionset sync complete: {target_table} (insert={ins_cnt}, update={upd_cnt})"
        )


    def _resolve_source_table(self, optionset_name: str, is_global: bool) -> Tuple[str, str]:
        opt = (optionset_name or "").lower()
        if opt == "statecode":
            return "StateMetadata", "State"
        if opt == "statuscode":
            return "StatusMetadata", "Status"
        if is_global:
            return "GlobalOptionsetMetadata", "Option"
        return "OptionsetMetadata", "Option"

    def _language_filter(self, df):
        if "LocalizedLabelLanguageId" not in df.columns:
            return df  # no language column -> no-op
        lang_col = "LocalizedLabelLanguageId"
        langs = [r[0] for r in df.select(lang_col).where(col(lang_col).isNotNull()).distinct().collect()]
        if not langs:
            return df
        chosen = 1033 if 1033 in langs else langs[0]
        return df.where(col(lang_col) == lit(chosen))


    def _read_bronze_options(self, entity_name: str, optionset_name: str, is_global: bool) -> DataFrame:
        source_table, value_col = self._resolve_source_table(optionset_name, is_global)
        src_full = f"{self.source_lakehouse}.{source_table}"
        df = spark.read.table(src_full)

        # Language smart-pick
        df = self._language_filter(df)

        # Filter by entity/optionset when present (entity ignored for globals)
        if entity_name and "EntityName" in df.columns:
            df = df.where(lower(col("EntityName")) == F.lit(entity_name.lower()))
        if optionset_name and "OptionSetName" in df.columns:
            df = df.where(lower(col("OptionSetName")) == F.lit(optionset_name.lower()))

        # Project: Name, SourceSystemId, and always expose Option for transforms
        return (
            df.select(
                substring(trim(col("LocalizedLabel")), 1, 500).alias("Name"),
                col(value_col).cast(StringType()).alias("SourceSystemId"),
                col(value_col).alias("Option"),  # unified for transforms
            )
            .dropna(subset=["Name", "SourceSystemId"])
            .dropDuplicates(["SourceSystemId"])
        )

    def _read_silver_state(self, target_table: str, target_pk: str) -> DataFrame:
        target_full  = f"{self.target_lakehouse}.{target_table}"
        mapping_full = self.mapping_table

        t = spark.read.table(target_full).alias("t")
        m = (
            spark.read.table(mapping_full).alias("m")
                .where((col("m.SourceId") == lit(self.source_id)) & (col("m.SourceTable") == lit(target_table)))
                .select(col("m.SilverRecordId"), col("m.SourceSystemId"))
        )
        return (
            t.join(m, col(f"t.{target_pk}") == col("m.SilverRecordId"), "inner")
             .select(
                 col(f"t.{target_pk}").alias("SilverRecordId"),
                 col("m.SourceSystemId").alias("SourceSystemId"),
                 col("t.Name").alias("Name")
             )
        )

    def _update_changed_options(self, bronze_opts: DataFrame, target_table: str, target_pk: str) -> int:
        target_full = f"{self.target_lakehouse}.{target_table}"
        silver_cur = self._read_silver_state(target_table, target_pk)
        creators = spark.read.table(target_full).select(
            col(target_pk).alias("SilverRecordId"),
            col("SourceId").alias("CreatorSourceId")
        )

        to_upd = (
            silver_cur.alias("s")
            .join(creators.alias("t"), col("s.SilverRecordId") == col("t.SilverRecordId"), "inner")
            .join(bronze_opts.alias("b"), col("s.SourceSystemId") == col("b.SourceSystemId"), "inner")
            .where(col("t.CreatorSourceId") == lit(self.source_id))
            .where(col("s.Name") != col("b.Name"))
            .select(
                col("s.SilverRecordId").alias(target_pk),
                col("b.Name").alias("Name"),
                current_timestamp().alias("ModifiedDate")
            )
        )

        cnt = to_upd.count()
        if cnt > 0:
            to_upd.createOrReplaceTempView("_opt_upd")
            spark.sql(f"""
                MERGE INTO {target_full} AS t
                USING _opt_upd AS s
                ON t.{target_pk} = s.{target_pk}
                WHEN MATCHED THEN UPDATE SET t.Name = s.Name, t.ModifiedDate = s.ModifiedDate
            """)
            spark.catalog.dropTempView("_opt_upd")
        return cnt

    def _insert_new_options(self, bronze_opts: DataFrame, target_table: str, target_pk: str) -> int:
        target_full = f"{self.target_lakehouse}.{target_table}"
        mapping_full = self.mapping_table
        tgt = spark.read.table(target_full)

        # Normalize names
        b_norm = bronze_opts.withColumn("__nm__", lower(trim(col("Name"))))

        # Deduplicate target by normalized name (choose earliest CreatedDate, else lowest PK)
        if "CreatedDate" in tgt.columns:
            t_norm_all = tgt.select(
                lower(trim(col("Name"))).alias("__nm__"),
                col(target_pk).alias("SilverRecordId"),
                col("CreatedDate")
            )
            w = Window.partitionBy("__nm__").orderBy(col("CreatedDate").asc_nulls_last())
        else:
            t_norm_all = tgt.select(
                lower(trim(col("Name"))).alias("__nm__"),
                col(target_pk).alias("SilverRecordId")
            ).withColumn("CreatedDate", lit(None).cast("timestamp"))
            w = Window.partitionBy("__nm__").orderBy(col("SilverRecordId").asc())

        t_norm = (
            t_norm_all
            .withColumn("_rn", row_number().over(w))
            .filter(col("_rn") == 1)
            .select("__nm__", "SilverRecordId")
        )

        # 3A) Name-share discovery → MERGE mappings 
        nm_pairs = (
            b_norm.alias("b").join(t_norm.alias("t"), on="__nm__", how="inner")
                 .select(
                     col("t.SilverRecordId").alias("SilverRecordId"),
                     lit(self.source_id).alias("SourceId"),
                     col("b.SourceSystemId").alias("SourceSystemId"),
                     lit(target_table).alias("SourceTable")
                 )
                 .dropDuplicates(["SilverRecordId","SourceId","SourceSystemId","SourceTable"])
        )
        nm_pairs.createOrReplaceTempView("_map_nm_pairs")

        spark.sql(f"""
            MERGE INTO {mapping_full} AS m
            USING _map_nm_pairs AS s
            ON  m.SilverRecordId = s.SilverRecordId
            AND m.SourceId       = s.SourceId
            AND m.SourceSystemId = s.SourceSystemId
            AND m.SourceTable    = s.SourceTable
            WHEN NOT MATCHED THEN
              INSERT (SilverRecordId, SourceId, SourceSystemId, SourceTable)
              VALUES (s.SilverRecordId, s.SourceId, s.SourceSystemId, s.SourceTable)
        """)
        spark.catalog.dropTempView("_map_nm_pairs")

        # 3B) Truly new → insert target + MERGE mappings
        new_rows = (
            b_norm.alias("b").join(t_norm.alias("t"), on="__nm__", how="left_anti")
                 .select(col("b.Name"), col("b.SourceSystemId"))
                 .dropDuplicates(["SourceSystemId"])   # safety: one per source code
        )

        to_ins = new_rows.select(
            expr("uuid()").alias(target_pk),
            col("Name"),
            current_timestamp().alias("CreatedDate"),
            current_timestamp().alias("ModifiedDate"),
            lit(self.source_id).alias("SourceId"),
            col("SourceSystemId")
        )

        cnt = to_ins.count()
        if cnt > 0:
            tgt_cols = set(tgt.schema.fieldNames())
            wanted = [target_pk, "Name", "CreatedDate", "ModifiedDate", "SourceId", "SourceSystemId"]
            available = [c for c in wanted if c in tgt_cols]
            to_ins.select(*available).write.mode("append").format("delta").saveAsTable(target_full)

            # Build mapping for newly inserted rows
            to_ins.createOrReplaceTempView("_map_new_rows")
            spark.sql(f"""
                MERGE INTO {mapping_full} AS m
                USING (
                  SELECT
                    {target_pk}     AS SilverRecordId,
                    '{self.source_id}' AS SourceId,
                    SourceSystemId   AS SourceSystemId,
                    '{target_table}' AS SourceTable
                  FROM _map_new_rows
                ) AS s
                ON  m.SilverRecordId = s.SilverRecordId
                AND m.SourceId       = s.SourceId
                AND m.SourceSystemId = s.SourceSystemId
                AND m.SourceTable    = s.SourceTable
                WHEN NOT MATCHED THEN
                  INSERT (SilverRecordId, SourceId, SourceSystemId, SourceTable)
                  VALUES (s.SilverRecordId, s.SourceId, s.SourceSystemId, s.SourceTable)
            """)
            spark.catalog.dropTempView("_map_new_rows")

        # 3C) BACKFILL: any target rows owned by this source without mapping 
        tgt_owned = tgt.select(col(target_pk).alias("SilverRecordId"),
                               col("SourceId"),
                               col("SourceSystemId")) \
                       .where(col("SourceId") == lit(self.source_id))

        existing_map_keys = spark.read.table(mapping_full) \
          .where((col("SourceId")==lit(self.source_id)) & (col("SourceTable")==lit(target_table))) \
          .select("SourceId","SourceSystemId","SourceTable") \
          .dropDuplicates()
        
        backfill = (
          tgt_owned
            .withColumn("SourceTable", lit(target_table))
            .select("SilverRecordId","SourceId","SourceSystemId","SourceTable")
            .join(existing_map_keys, on=["SourceId","SourceSystemId","SourceTable"], how="left_anti")
        )


        if not backfill.rdd.isEmpty():
            backfill.createOrReplaceTempView("_map_backfill")
            spark.sql(f"""
                MERGE INTO {mapping_full} AS m
                USING _map_backfill AS s
                ON  m.SilverRecordId = s.SilverRecordId
                AND m.SourceId       = s.SourceId
                AND m.SourceSystemId = s.SourceSystemId
                AND m.SourceTable    = s.SourceTable
                WHEN NOT MATCHED THEN
                  INSERT (SilverRecordId, SourceId, SourceSystemId, SourceTable)
                  VALUES (s.SilverRecordId, s.SourceId, s.SourceSystemId, s.SourceTable)
            """)
            spark.catalog.dropTempView("_map_backfill")

        return cnt
        
 #   def _delete_removed_options(self, bronze_opts: DataFrame, target_table: str, target_pk: str) -> int:
 #       """Optional hard-delete for optionset rows owned by this SourceId and unmapped in Bronze.
 #       Disabled by default (sticky mode). Enable via enable_delete=True
 #       """
 #       target_full = f"{self.target_lakehouse}.{target_table}"
 #       mapping_full = self.mapping_table
 #       silver_cur = self._read_silver_state(target_table, target_pk)
 #
 #       to_delete = (
 #           silver_cur.alias("s")
 #           .join(bronze_opts.alias("b"), col("s.SourceSystemId") == col("b.SourceSystemId"), "left_anti")
 #           .select("SilverRecordId")
 #       )
 #       cnt = to_delete.count()
 #       if cnt == 0:
 #           return 0
 #
 #       logging.info(f"🗑️ Deleting {cnt} record(s) from {target_table} and mapping (scoped to this source)")
 #       to_delete.createOrReplaceTempView("_opt_del")
 #       # Delete target rows
 #       spark.sql(f"""
 #           DELETE FROM {target_full}
 #           WHERE {target_pk} IN (SELECT SilverRecordId FROM _opt_del)
 #       """)
 #       # Delete mappings
 #       spark.sql(f"""
 #           DELETE FROM {mapping_full}
 #           WHERE SilverRecordId IN (SELECT SilverRecordId FROM _opt_del)
 #       """)
 #       spark.catalog.dropTempView("_opt_del")
 #       return cnt
 #

## Dynamics 365 watermark

Service class for managing watermark state tracking for incremental synchronization.

In [ ]:
from typing import Optional
from datetime import datetime, timezone
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, TimestampType
import logging

class Dynamics365Watermark:
    """
    Service for managing watermark state tracking against a table with columns:
      - source_table_name STRING
      - target_table_name STRING
      - ts_column          STRING
      - last_processed_ts  TIMESTAMP
      - updated_at         TIMESTAMP
    """
    WATERMARK_TABLE = "WatermarkState"

    def __init__(self, source_lakehouse: str, target_lakehouse: str):
        self.source_lakehouse = source_lakehouse
        self.target_lakehouse = target_lakehouse
        self.watermark_table_full = f"{target_lakehouse}.{self.WATERMARK_TABLE}"
        self._ts_column: Optional[str] = None  # set via set_ts_column()

    # --- optional helper so you can persist the name of the watermark column used (e.g., "SinkModifiedOn")
    def set_ts_column(self, ts_col_name: str):
        self._ts_column = ts_col_name

    def ensure_watermark_table(self):
        """Create WatermarkState table if it doesn't exist (with your schema)."""
        if self._table_exists(self.watermark_table_full):
            logging.debug(f"ℹ️ Watermark table already exists: {self.watermark_table_full}")
            return

        schema = StructType([
            StructField("source_table_name", StringType(), True),
            StructField("target_table_name", StringType(), True),
            StructField("ts_column",          StringType(), True),
            StructField("last_processed_ts",  TimestampType(), True),
            StructField("updated_at",         TimestampType(), True)
        ])
        empty_df = spark.createDataFrame([], schema)
        empty_df.write.mode("overwrite").format("delta").saveAsTable(self.watermark_table_full)
        logging.info(f"✅ Created watermark table: {self.watermark_table_full}")

    def get_last_watermark(self, source_table: str, target_table: str):
        """Return the last processed watermark (Python datetime) or None."""
        if not self._table_exists(self.watermark_table_full):
            self.ensure_watermark_table()
            return None

        df = (spark.read.table(self.watermark_table_full)
                    .filter(col("source_table_name") == source_table)
                    .filter(col("target_table_name") == target_table))

        # Robust emptiness check
        if df.head(1) == []:
            return None

        return df.select(F.max("last_processed_ts").alias("ts")).first()["ts"]

    def get_current_watermark(self, source_table: str, watermark_column: str, source_table_lakehouse: Optional[str] = None):
        """
        Return max(watermark_column) from Bronze as a Python datetime (fallback to now UTC).
        
        Args:
            source_table: Table name
            watermark_column: Column to read max value from
            source_table_lakehouse: Lakehouse prefix override. If None, uses configured Bronze lakehouse.
                                   If empty string "", reads without prefix (for TEMPORARY VIEW).
        """
        # Determine lakehouse prefix
        if source_table_lakehouse is None:
            lakehouse_prefix = self.source_lakehouse
        else:
            lakehouse_prefix = source_table_lakehouse
        
        # Build full table name
        # NOTE: Cannot use f"{lakehouse_prefix}.{source_table}" directly because when lakehouse_prefix=""
        # it would produce ".source_table" (starts with dot) which is invalid table name.
        # Must conditionally add dot only when prefix exists to support TEMPORARY VIEWs.
        # When reading from temporary view (created via createOrReplaceTempView), we need to read without
        # lakehouse prefix, just the view name itself (e.g., "address_stage_union" not "lakehouse.address_stage_union").
        # In Python, empty string "" is falsy, so `if lakehouse_prefix:` evaluates to False when lakehouse_prefix="",
        # causing the code to go to else branch and use just source_table without prefix.
        if lakehouse_prefix:
            source_full = f"{lakehouse_prefix}.{source_table}"
        else:
            source_full = source_table
        
        row = (spark.read.table(source_full)
                    .select(F.max(col(watermark_column)).alias("max_wm"))
                    .first())
        if row is not None and row["max_wm"] is not None:
            return row["max_wm"]
        return datetime.now(timezone.utc)

    def update_watermark(self, source_table: str, target_table: str, watermark):
        """Upsert the watermark row using only Python literals (no Column objects)."""
        self.ensure_watermark_table()

        # Normalize to Python datetimes
        if not isinstance(watermark, datetime):
            watermark = spark.sql("SELECT current_timestamp() AS ts").first()["ts"]
        now_py = spark.sql("SELECT current_timestamp() AS ts").first()["ts"]

        ts_col_val = self._ts_column if self._ts_column is not None else ""

        src = spark.createDataFrame(
            [(source_table, target_table, ts_col_val, watermark, now_py)],
            ["source_table_name", "target_table_name", "ts_column", "last_processed_ts", "updated_at"]
        )
        src.createOrReplaceTempView("_watermark_update")

        spark.sql(f"""
            MERGE INTO {self.watermark_table_full} AS t
            USING _watermark_update AS s
              ON t.source_table_name = s.source_table_name
             AND t.target_table_name = s.target_table_name
            WHEN MATCHED THEN UPDATE SET
                t.ts_column         = CASE WHEN s.ts_column = '' THEN t.ts_column ELSE s.ts_column END,
                t.last_processed_ts = s.last_processed_ts,
                t.updated_at        = s.updated_at
            WHEN NOT MATCHED THEN INSERT (
                source_table_name, target_table_name, ts_column, last_processed_ts, updated_at
            ) VALUES (
                s.source_table_name, s.target_table_name, s.ts_column, s.last_processed_ts, s.updated_at
            )
        """)
        spark.catalog.dropTempView("_watermark_update")

    def _table_exists(self, table_name: str) -> bool:
        """Check if table exists."""
        try:
            spark.read.table(table_name).limit(1)
            return True
        except Exception:
            return False


## Dynamics 365 data synchronization

Main orchestration class for Dynamics 365 Bronze-to-Silver synchronization.

**Architecture**: Uses composition pattern with three specialized components:
- `Dynamics365DataReader` - Reads data from Bronze lakehouse
- `Dynamics365ForeignKeyResolver` - Resolves Bronze FKs to Silver GUIDs
- `Dynamics365DataWriter` - Writes data to Silver lakehouse

**Features**:
- Watermark-based incremental synchronization
- Automatic GUID generation for Silver IDs
- SourceSystemIdMapping maintenance
- Hard delete support with optional policy
- Custom transformation functions
- Automatic foreign key resolution via `fk_mappings`


### Dynamics 365 data reader

Component for reading data from Bronze lakehouse with incremental sync support.

In [ ]:
from typing import List, Optional
from pyspark.sql import DataFrame, Window
from pyspark.sql.functions import col, lit, row_number, trim, lower
from pyspark.sql.types import StringType, TimestampType
import logging


class Dynamics365DataReader:
    """
    Component responsible for reading data from Bronze lakehouse.
    
    Capabilities:
    - Incremental reading based on watermark column
    - Bronze lookup table reads with column mapping and casting
    - Activity party reads (Dynamics 365 standard pattern)
    - Extract deletes vs active records
    - Deduplication of multiple modifications
    """
    
    def __init__(self, source_lakehouse: str, watermark_column: str = "SinkModifiedOn", delete_flag_column: str = "IsDelete"):
        """
        Initialize data reader component.
        
        Args:
            source_lakehouse: Bronze lakehouse name
            watermark_column: Column used for incremental sync (default: "SinkModifiedOn")
            delete_flag_column: Column indicating deleted records (default: "IsDelete")
        """
        self.source_lakehouse = source_lakehouse
        self.watermark_column = watermark_column
        self.delete_flag_column = delete_flag_column
    
    def read_incremental_changes(
        self,
        source_table: str,
        source_primary_key: str,
        source_columns: List[str],
        last_watermark: Optional[str],
        current_watermark: str,
        source_table_lakehouse: Optional[str] = None
    ) -> DataFrame:
        """
        Load only records that changed since last sync run (incremental pattern).
        
        Instead of reading millions of records every time, we use timestamps
        to find just the new/modified ones. Much faster and cheaper!
        
        Args:
            source_table: Bronze table name
            source_primary_key: Primary key column name
            source_columns: Columns to read
            last_watermark: Last sync watermark (None for initial load)
            current_watermark: Current watermark (upper bound)
            source_table_lakehouse: Lakehouse prefix for source_table. If None, uses self.source_lakehouse.
                                   Set to empty string "" to read without any prefix (for TEMPORARY VIEW).
        
        Returns:
            DataFrame with records in watermark window
        """
        # Determine lakehouse prefix
        if source_table_lakehouse is None:
            # Default: use configured Bronze lakehouse
            source_full = f"{self.source_lakehouse}.{source_table}"
        elif source_table_lakehouse == "":
            # Empty string: no prefix (for TEMPORARY VIEW)
            source_full = source_table
        else:
            # Custom lakehouse specified
            source_full = f"{source_table_lakehouse}.{source_table}"
        
        df = spark.read.table(source_full)
        
        # Only read records modified between last sync and now (time window filtering)
        if last_watermark:
            df = df.filter(col(self.watermark_column) > lit(last_watermark))
        df = df.filter(col(self.watermark_column) <= lit(current_watermark))
        
        # Pick only the columns we need (saves memory and processing time)
        required_cols = list(set(
            source_columns +
            [source_primary_key, self.watermark_column] +
            ([self.delete_flag_column] if self.delete_flag_column else [])
        ))
        
        return df.select(*required_cols)
    
    def extract_deletes(self, df: DataFrame, source_primary_key: str) -> DataFrame:
        """
        Find records marked as deleted in the source system.
        
        Source systems usually don't physically delete records - they flag them
        with IsDelete=1. We identify these so we can remove them from Silver.
        
        Args:
            df: DataFrame from incremental read
            source_primary_key: Primary key column name
        
        Returns:
            DataFrame with only deleted record IDs
        """
        if not self.delete_flag_column:
            return spark.createDataFrame([], df.schema)
        
        return df.filter(col(self.delete_flag_column) == 1) \
            .select(source_primary_key) \
            .dropDuplicates()
    
    def extract_active(self, df: DataFrame, source_primary_key: str) -> DataFrame:
        """
        Get records that should exist in Silver (not deleted).
        
        We also keep only the LATEST version if a record was modified multiple times
        since last sync. No point processing the same record 5 times.
        
        Args:
            df: DataFrame from incremental read
            source_primary_key: Primary key column name
        
        Returns:
            Deduplicated DataFrame with only active (non-deleted) records
        """
        if self.delete_flag_column:
            df = df.filter((col(self.delete_flag_column).isNull()) | 
                          (col(self.delete_flag_column) == 0))
        
        # If a record was modified 3 times, keep only the newest version (deduplication)
        # Secondary sort by PK ensures deterministic behavior when watermarks collide
        w = Window.partitionBy(source_primary_key) \
            .orderBy(col(self.watermark_column).desc(), col(source_primary_key).asc())
        
        result = df.withColumn("rn", row_number().over(w)) \
            .filter(col("rn") == 1) \
            .drop("rn", self.watermark_column)
        
        if self.delete_flag_column:
            result = result.drop(self.delete_flag_column)
            
        return result
    
    def read_bronze_table(
        self,
        table_name: str,
        columns,  # Union[List[str], Dict]
        filters: Optional[List] = None
    ) -> DataFrame:
        """
        Read Bronze table with column mapping, casting, and filtering.
        
        Universal Bronze reader for all Bronze table reads (lookups, full entities, staging).
        Supports column renaming, type casting, and filtering. Auto-trims string columns.
        
        Args:
            table_name: Bronze table name (without lakehouse prefix)
            columns: Column specification - can be:
                    1) List of column names (keeps original names): ["col1", "col2", "col3"]
                    2) Dict with rename/cast: {"source_col": "alias"} or {"source_col": ("alias", "cast_type")}
                    3) Mix in dict: {"col1": "col1", "col2": ("col2_renamed", "string")}
                    
                    Special handling for dict values:
                    - String: Simple rename/passthrough
                    - Tuple ("alias", "string"): Applies trim() before casting
                    - Tuple ("alias", "timestamp"): Casts to TimestampType
                    
            filters: Optional list of filter conditions (Column expressions)
        
        Returns:
            DataFrame with selected/renamed/cast columns
        
        Examples:
            # List format - keeps original column names (no repetition!)
            contact = reader.read_bronze_table("contact", [
                "address1_city", "address1_country", "createdon", "modifiedon"
            ])
            
            # Dict format - rename/cast
            wf = reader.read_bronze_table("workflow", {
                "workflowid": "wf_id",
                "name": ("wf_name", "string")  # auto-trims
            })
            
            # Mixed format - some passthrough, some with cast
            tx = reader.read_bronze_table("msnfp_transaction", {
                "Id": "Id",  # passthrough
                "createdon": ("createdon", "timestamp"),  # cast
                "SinkModifiedOn": ("SinkModifiedOn", "timestamp")
            })
        """
        # Read table from Bronze lakehouse
        df = spark.read.table(f"{self.source_lakehouse}.{table_name}")
        
        # Apply filters first (reduces data before projections)
        if filters:
            for condition in filters:
                df = df.filter(condition)
        
        # Handle list format (simple column names)
        if isinstance(columns, list):
            return df.select(*[col(c) for c in columns])
        
        # Handle dict format (rename/cast)
        select_exprs = []
        for src_col, spec in columns.items():
            if isinstance(spec, tuple):
                # Tuple format: ("alias", "cast_type")
                alias, cast_type = spec
                expr_col = col(src_col)
                
                # Auto-trim strings before casting
                if cast_type == "string":
                    expr_col = trim(expr_col).cast(StringType())
                elif cast_type == "timestamp":
                    expr_col = expr_col.cast(TimestampType())
                else:
                    # Generic cast (int, bigint, decimal, etc.)
                    expr_col = expr_col.cast(cast_type)
                
                expr_col = expr_col.alias(alias)
            else:
                # Simple string format: just alias
                expr_col = col(src_col).alias(spec)
            
            select_exprs.append(expr_col)
        
        return df.select(*select_exprs)
    
    def read_activity_parties(
        self,
        activity_id_column: str = "activityid",
        participation_types: list = [1, 2]
    ) -> DataFrame:
        """
        Read activityparty records from Bronze lakehouse (Dynamics 365 standard pattern).
        
        This is a Dynamics 365 universal pattern - every activity (letter, phonecall, email, appointment, fax)
        has activityparty records that define participants (from/to/CC/BCC).
        
        Args:
            activity_id_column: Column name for activity ID (default: "activityid")
            participation_types: List of participationtypemask values to include
                               Default [1, 2] = from/to
                               Dynamics 365 values: 1=From, 2=To, 3=CC, 4=BCC, 5=Required, 6=Optional, 7=Organizer, etc.
        
        Returns:
            DataFrame with columns:
            - activityid: The activity ID (cast to string)
            - partyid: The party ID (contact/account - cast to string)
            - partyid_entitytype: The party entity type (lowercased)
            - ActivityPartyId: The activityparty unique ID (cast to string)
        
        Example usage:
            parties = reader.read_activity_parties(participation_types=[1, 2, 3])
            df = df.join(parties, col("Id") == col("activityid"), "inner")
        """
        activityparty = (
            spark.read.table(f"{self.source_lakehouse}.activityparty")
            .filter(col("participationtypemask").isin(participation_types))
            .filter(col("partyid").isNotNull())
            .filter(lower(col("partyid_entitytype")).isin("contact", "account"))
            .select(
                col(activity_id_column).alias("activityid"),
                col("partyid").cast(StringType()).alias("partyid"),
                lower(col("partyid_entitytype")).alias("partyid_entitytype"),
                col("Id").cast(StringType()).alias("ActivityPartyId")
            )
            .dropDuplicates(["activityid", "partyid", "partyid_entitytype", "ActivityPartyId"])
        )
        
        logging.info(f"📋 Read activity parties (participation types: {participation_types})")
        return activityparty

### Dynamics 365 foreign key resolver

Component for resolving foreign keys from Bronze IDs to Silver GUIDs.

In [ ]:
from typing import Dict, Optional
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit, lower, coalesce as F_coalesce
from pyspark.sql.types import StringType
import pyspark.sql.functions as F
import logging


class Dynamics365ForeignKeyResolver:
    """
    Component responsible for resolving foreign keys from Bronze to Silver.
    
    Capabilities:
    - Standard FK resolution (Bronze ID → Silver GUID via SourceSystemIdMapping)
    - Constituent resolution (Contact/Account → Constituent)
    - Polymorphic lookup resolution (regardingobjectid, partyid, etc.)
    """
    
    def __init__(self, source_id: str, source_lakehouse: str, target_lakehouse: str, mapping_table: str):
        """
        Initialize FK resolver component.
        
        Args:
            source_id: SourceId GUID for this data source
            source_lakehouse: Bronze lakehouse name
            target_lakehouse: Silver lakehouse name
            mapping_table: Full path to SourceSystemIdMapping table
        """
        self.source_id = source_id
        self.source_lakehouse = source_lakehouse
        self.target_lakehouse = target_lakehouse
        self.mapping_table = mapping_table
    
    def resolve_foreign_keys(self, df: DataFrame, fk_mappings: Dict[str, str]) -> DataFrame:
        """
        Convert source system codes to Silver GUIDs for related records.
        
        WHY: Source data uses numeric codes (e.g., typecode=1 means "Email Campaign").
        Silver layer uses unique GUIDs for each record. This method translates between them.
        
        EXAMPLE: 
        - Input: Contact with address1_addressid=123 (source system code)
        - Output: Same contact, but also with address1_addressid_SilverRecordId=abc-def-guid
        - Now we can link Contact → Address in Silver using the GUID
        
        This is a PUBLIC method you can call from custom transform functions
        if you need to resolve additional relationships.
        
        Args:
            df: DataFrame with source data
            fk_mappings: Which columns link to which tables
                        Example: {"typecode": "CampaignType", "address1_addressid": "Address"}
        
        Returns:
            Original DataFrame plus new {column}_SilverRecordId columns for each mapping
        """
        result_df = df
        
        for bronze_col, silver_table in fk_mappings.items():
            if bronze_col not in df.columns:
                logging.warning(f"⚠️ FK column '{bronze_col}' not found in DataFrame, skipping resolution")
                continue
            
            logging.info(f"🔗 Resolving FK: {bronze_col} → {silver_table}")
            
            # Read mapping for target Silver table
            # DEFENSIVE: Deduplicate by SourceSystemId to prevent JOIN creating duplicates.
            # Ideally (SourceId, SourceSystemId, SourceTable) should have UNIQUE constraint,
            # but without it we must deduplicate to avoid MERGE error when corrupted mappings exist.
            # If duplicates exist, we arbitrarily pick first (non-deterministic but safe).
            mapping = spark.read.table(self.mapping_table) \
                .filter(col("SourceId") == lit(self.source_id)) \
                .filter(col("SourceTable") == lit(silver_table)) \
                .select(
                    col("SilverRecordId"),
                    col("SourceSystemId").alias(f"_fk_{bronze_col}_ssid")
                ) \
                .dropDuplicates([f"_fk_{bronze_col}_ssid"])
            
            # Join to resolve Bronze value → Silver GUID
            result_df = result_df.join(
                mapping,
                col(bronze_col).cast(StringType()) == col(f"_fk_{bronze_col}_ssid"),
                "left"
            ) \
            .withColumnRenamed("SilverRecordId", f"{bronze_col}_SilverRecordId") \
            .drop(f"_fk_{bronze_col}_ssid")
            
            # Count resolution statistics
            non_null_source = result_df.filter(col(bronze_col).isNotNull()).count()
            resolved_count = result_df.filter(col(f"{bronze_col}_SilverRecordId").isNotNull()).count()
            
            # Determine log message based on resolution success
            if non_null_source == 0:
                logging.info(f"   ℹ️ All records have NULL {bronze_col} (optional FK)")
            elif resolved_count == non_null_source:
                logging.info(f"   ✓ Resolved all {resolved_count} {bronze_col} references")
            else:
                unresolved = non_null_source - resolved_count
                logging.warning(f"   ⚠️ Resolved only {resolved_count} of {non_null_source} {bronze_col} references ({unresolved} not found in {silver_table})")
        
        return result_df
    
    def resolve_constituent_id(
        self,
        df: DataFrame,
        contact_fk_column: Optional[str] = None,
        account_fk_column: Optional[str] = None,
        output_column: str = "ConstituentId"
    ) -> DataFrame:
        """
        Resolve ConstituentId from Contact or Account foreign keys.
        
        Constituent is an abstraction over Contact and Account entities.
        This method joins the Constituent table via ContactId or AccountId
        to find the corresponding ConstituentId.
        
        USAGE PATTERN:
        1. Framework resolves Contact/Account FKs first (via fk_mappings)
        2. Call this method to resolve ConstituentId from those resolved FKs
        3. Prefer Contact over Account when both are present (coalesce)
        
        EXAMPLE:
            # In transform_transaction:
            def transform_transaction(df: DataFrame) -> DataFrame:
                # Framework already resolved:
                # - msnfp_receiptoncontactid → msnfp_receiptoncontactid_SilverRecordId
                # - msnfp_transaction_receiptonaccountid → msnfp_transaction_receiptonaccountid_SilverRecordId
                
                # Now resolve Constituent:
                df = resolver.resolve_constituent_id(
                    df,
                    contact_fk_column="msnfp_receiptoncontactid_SilverRecordId",
                    account_fk_column="msnfp_transaction_receiptonaccountid_SilverRecordId"
                )
                
                return df.select(..., col("ConstituentId"), ...)
        
        Args:
            df: DataFrame with resolved Contact/Account FK columns
            contact_fk_column: Name of column containing resolved ContactId (FK to Contact)
            account_fk_column: Name of column containing resolved AccountId (FK to Account)
            output_column: Name of output column (default: "ConstituentId")
        
        Returns:
            Original DataFrame plus new {output_column} with resolved ConstituentId.
            Prefers Contact path over Account path when both are present.
        """
        if not contact_fk_column and not account_fk_column:
            raise ValueError("Must provide at least one of contact_fk_column or account_fk_column")
        
        logging.info(f"🔗 Resolving {output_column} from Constituent table")
        
        # Read only needed columns from Constituent table (optimized)
        constituent = spark.read.table(f"{self.target_lakehouse}.Constituent") \
            .select("ConstituentId", "ContactId", "AccountId")
        
        result_df = df
        
        # Resolve via Contact path
        if contact_fk_column and contact_fk_column in df.columns:
            cons_contact = constituent.select(
                col("ConstituentId").alias("_cons_from_contact"),
                col("ContactId").alias("_cons_contact_key")
            )
            
            result_df = result_df.join(
                cons_contact,
                col(contact_fk_column) == col("_cons_contact_key"),
                "left"
            ).drop("_cons_contact_key")
        
        # Resolve via Account path
        if account_fk_column and account_fk_column in df.columns:
            cons_account = constituent.select(
                col("ConstituentId").alias("_cons_from_account"),
                col("AccountId").alias("_cons_account_key")
            )
            
            result_df = result_df.join(
                cons_account,
                col(account_fk_column) == col("_cons_account_key"),
                "left"
            ).drop("_cons_account_key")
        
        # Coalesce: prefer Contact over Account
        if contact_fk_column and account_fk_column:
            result_df = result_df.withColumn(
                output_column,
                F_coalesce(col("_cons_from_contact"), col("_cons_from_account"))
            )
        elif contact_fk_column:
            result_df = result_df.withColumnRenamed("_cons_from_contact", output_column)
        else:  # account_fk_column only
            result_df = result_df.withColumnRenamed("_cons_from_account", output_column)
        
        # Cleanup temporary columns
        temp_cols = ["_cons_from_contact", "_cons_from_account"]
        for temp_col in temp_cols:
            if temp_col in result_df.columns:
                result_df = result_df.drop(temp_col)
        
        # Single count operation for logging (avoid multiple Spark evaluations)
        total_resolved = result_df.filter(col(output_column).isNotNull()).count()
        total_count = result_df.count()
        
        # Better messaging: distinguish between successful resolution and missing data
        if total_resolved == total_count:
            logging.info(f"   ✓ Resolved all {total_resolved} Constituents")
        else:
            null_count = total_count - total_resolved
            logging.info(f"   ✓ Resolved {total_resolved} Constituents ({null_count} records without Contact/Account link)")
        
        return result_df
    
    def resolve_polymorphic_lookup(
        self,
        df: DataFrame,
        lookup_id_column: str,
        entity_type_column: str,
        entity_type_mappings: dict
    ) -> DataFrame:
        """
        Resolve Dynamics 365 polymorphic lookup fields (customerid + customerid_entitytype pattern).
        
        Dynamics 365 uses polymorphic lookups where one field can reference different entity types:
        - lookup_id_column: The ID value (e.g., "customerid")
        - entity_type_column: Entity type discriminator (e.g., "customerid_entitytype")
        - entity_type_mappings: Dict mapping entity type values to Silver table names
          Example: {"contact": "Contact", "account": "Account"}
        
        Returns:
            DataFrame with resolved FK columns added for each entity type.
            Column naming: {entity_type_value}Id (e.g., "ContactId", "AccountId")
        
        Example usage:
            df = resolver.resolve_polymorphic_lookup(
                df,
                lookup_id_column="customerid",
                entity_type_column="customerid_entitytype",
                entity_type_mappings={"contact": "Contact", "account": "Account"}
            )
            # Adds: ContactId and AccountId columns
        """
        if not entity_type_mappings:
            raise ValueError("entity_type_mappings cannot be empty")
        
        logging.info(f"🔗 Resolving polymorphic lookup: {lookup_id_column} by {entity_type_column}")
        
        result_df = df
        
        # Resolve each entity type
        for entity_type_value, silver_table in entity_type_mappings.items():
            # Get mapping for this entity type
            mapping = (
                spark.read.table(self.mapping_table)
                .filter((col("SourceId") == lit(self.source_id)) & 
                        (lower(col("SourceTable")) == entity_type_value.lower()))
                .select(
                    col("SourceSystemId").alias(f"_src_{entity_type_value}"),
                    col("SilverRecordId").alias(f"{entity_type_value.capitalize()}Id")
                )
            )
            
            # Join when entity type matches
            result_df = result_df.join(
                mapping,
                (col(lookup_id_column).cast(StringType()) == col(f"_src_{entity_type_value}")) & 
                (lower(col(entity_type_column)) == entity_type_value.lower()),
                "left"
            ).drop(f"_src_{entity_type_value}")
            
            logging.info(f"   ✓ Resolved {entity_type_value} → {entity_type_value.capitalize()}Id")
        
        return result_df

### Dynamics 365 data writer

Component for writing data to Silver lakehouse with MERGE operations and mapping management.

In [ ]:
from typing import Optional, Callable
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit, expr, coalesce
from pyspark.sql.types import StringType
import logging


class Dynamics365DataWriter:
    """
    Component responsible for writing data to Silver lakehouse.
    
    Capabilities:
    - Process deletes (with optional delete policy)
    - Process upserts (INSERT/UPDATE via MERGE)
    - Generate Silver GUIDs for new records
    - Maintain SourceSystemIdMapping table
    - Support custom MERGE SQL
    """
    
    def __init__(self, source_id: str, target_lakehouse: str, mapping_table: str, enable_hard_delete: bool = True):
        """
        Initialize data writer component.
        
        Args:
            source_id: SourceId GUID for this data source
            target_lakehouse: Silver lakehouse name
            mapping_table: Full path to SourceSystemIdMapping table
            enable_hard_delete: Whether to process hard deletes (default: True)
        """
        self.source_id = source_id
        self.target_lakehouse = target_lakehouse
        self.mapping_table = mapping_table
        self.enable_hard_delete = enable_hard_delete
    
    def process_deletes(
        self,
        df_deletes: DataFrame,
        source_primary_key: str,
        target_table: str,
        target_primary_key: str,
        delete_policy_func: Optional[Callable[[DataFrame], DataFrame]] = None
    ):
        """
        Remove deleted records from Silver layer.
        
        Sometimes you don't want to delete everything (e.g., keep historical data).
        That's what delete_policy_func is for - it can filter which deletes to actually execute.
        
        Args:
            df_deletes: DataFrame with deleted record IDs (from Bronze)
            source_primary_key: Bronze PK column name
            target_table: Silver table name
            target_primary_key: Silver PK column name
            delete_policy_func: Optional function to filter deletes before execution
        """
        if not self.enable_hard_delete or df_deletes.isEmpty():
            return
        
        delete_count = df_deletes.count()
        logging.info(f"🗑️ Processing {delete_count} delete(s) for {target_table}")
        
        # Get mapping to resolve Bronze PK → Silver PK
        mapping_df = self._get_mapping_for_deletes(df_deletes, source_primary_key, target_table, target_primary_key)
        
        if mapping_df.isEmpty():
            logging.info(f"ℹ️ No matching Silver records to delete")
            return
        
        # Apply delete policy if provided
        if delete_policy_func:
            before_count = mapping_df.count()
            mapping_df = delete_policy_func(mapping_df)
            after_count = mapping_df.count()
            excluded = before_count - after_count
            if excluded > 0:
                logging.info(f"🛡️ Delete policy excluded {excluded} record(s)")
        
        # Execute delete
        if not mapping_df.isEmpty():
            self._execute_delete(mapping_df, target_table, target_primary_key)
    
    def _get_mapping_for_deletes(
        self,
        df_deletes: DataFrame,
        source_primary_key: str,
        target_table: str,
        target_primary_key: str
    ) -> DataFrame:
        """
        Find Silver GUIDs for the source records we need to delete.
        
        Source says "delete record ID=123". We look up which Silver GUID
        corresponds to that source ID, so we know what to remove.
        """
        mapping = spark.read.table(self.mapping_table) \
            .filter(col("SourceId") == lit(self.source_id)) \
            .filter(col("SourceTable") == lit(target_table)) \
            .select(
                col("SilverRecordId").alias(target_primary_key),
                col("SourceSystemId")
            )
        
        return df_deletes \
            .withColumn("SourceSystemId", col(source_primary_key).cast(StringType())) \
            .join(mapping, "SourceSystemId", "inner") \
            .select(target_primary_key)
    
    def _execute_delete(self, keys_df: DataFrame, target_table: str, target_primary_key: str):
        """
        Physically remove records from Silver and clean up the ID mapping table.
        
        We delete from TWO places:
        1. The target table (e.g., Campaign table)
        2. The mapping table (which tracks source ID → Silver GUID relationships)
        """
        keys_df.createOrReplaceTempView("_delete_keys_temp")
        target_full = f"{self.target_lakehouse}.{target_table}"
        
        # 1. Delete from target table (e.g., Campaign)
        delete_sql = f"""
        MERGE INTO {target_full} AS t
        USING _delete_keys_temp AS k
          ON t.{target_primary_key} = k.{target_primary_key}
        WHEN MATCHED THEN DELETE
        """
        
        spark.sql(delete_sql)
        logging.info(f"✓ Deleted records from {target_table}")
        
        # 2. Delete corresponding mappings from SourceSystemIdMapping
        mapping_delete_sql = f"""
        MERGE INTO {self.mapping_table} AS m
        USING _delete_keys_temp AS k
          ON m.SilverRecordId = k.{target_primary_key}
         AND m.SourceId = '{self.source_id}'
         AND m.SourceTable = '{target_table}'
        WHEN MATCHED THEN DELETE
        """
        
        spark.sql(mapping_delete_sql)
        logging.info(f"✓ Deleted corresponding mappings from SourceSystemIdMapping")
        
        spark.catalog.dropTempView("_delete_keys_temp")
    
    def process_upserts(
        self,
        df_active: DataFrame,
        source_primary_key: str,
        target_table: str,
        target_primary_key: str,
        transform_func: Optional[Callable[[DataFrame], DataFrame]] = None,
        merge_sql: Optional[str] = None,
        mapping_table_name: Optional[str] = None
    ):
        """
        Add new records and update existing ones in Silver.
        
        "Upsert" = UPDATE if exists, INSERT if new. Smart merge that handles both cases.
        This is the core of data synchronization - keeping Silver in sync with source.
        
        Args:
            df_active: DataFrame with active (non-deleted) records
            source_primary_key: Bronze PK column name
            target_table: Silver table name
            target_primary_key: Silver PK column name
            transform_func: Optional transformation function
            merge_sql: Optional custom MERGE SQL
            mapping_table_name: Override SourceTable value in mapping
        """
        if df_active.isEmpty():
            return
        
        upsert_count = df_active.count()
        logging.info(f"➕ Processing {upsert_count} upsert(s) for {target_table}")
        
        # 1. Transform data (user transformation)
        if transform_func:
            df_transformed = transform_func(df_active)
        else:
            df_transformed = df_active
        
        # 2. Add required Silver columns (SourceId, SourceSystemId) if not present
        df_transformed = self._ensure_required_columns(df_transformed, source_primary_key)
        
        # 3. Resolve existing records (UPDATE) vs new records (INSERT)
        df_with_mapping = self._resolve_silver_ids(df_transformed, target_table)
        
        # 4. Generate GUIDs for new records
        df_final = self._generate_silver_ids(df_with_mapping, target_primary_key)
        
        # 5. Execute MERGE (custom or auto-generated)
        self._execute_upsert(df_final, target_table, target_primary_key, merge_sql)
        
        # 6. Update SourceSystemIdMapping
        self._update_mapping(df_final, target_table, target_primary_key, mapping_table_name)
    
    def _ensure_required_columns(self, df: DataFrame, source_primary_key: str) -> DataFrame:
        """
        Add tracking columns that every Silver record needs.
        
        Every record in Silver must have:
        - SourceId: Which source system it came from (this Dynamics instance)
        - SourceSystemId: The original ID from that source system
        
        This lets us trace any Silver record back to its origin and handle updates correctly.
        Transform functions don't need to add these - we do it automatically.
        """
        result_df = df
        
        # Add SourceId if not present
        if "SourceId" not in df.columns:
            result_df = result_df.withColumn("SourceId", lit(self.source_id))
        
        # Add SourceSystemId if not present
        if "SourceSystemId" not in df.columns:
            result_df = result_df.withColumn("SourceSystemId", col(source_primary_key).cast(StringType()))
        
        return result_df
    
    def _resolve_silver_ids(self, df: DataFrame, target_table: str) -> DataFrame:
        """
        Check which records already exist in Silver vs. which are brand new.
        
        We look up each source ID in our mapping table. If found, we get back
        the Silver GUID to UPDATE. If not found, it's a new record to INSERT.
        """
        mapping = spark.read.table(self.mapping_table) \
            .filter(col("SourceId") == lit(self.source_id)) \
            .filter(col("SourceTable") == lit(target_table)) \
            .select(
                col("SilverRecordId"),
                col("SourceSystemId").alias("_map_SourceSystemId")
            )
        
        joined = df.join(
            mapping,
            (df["SourceId"] == lit(self.source_id)) &
            (df["SourceSystemId"] == col("_map_SourceSystemId")),
            "left"
        ).drop("_map_SourceSystemId")  
        return joined
    
    def _generate_silver_ids(self, df: DataFrame, target_primary_key: str) -> DataFrame:
        """
        Create unique Silver GUIDs for new records.
        
        Existing records keep their Silver GUID (from mapping table).
        New records get a fresh UUID. This ensures every Silver record
        has a stable, unique identifier across all syncs.
        """
        return df.withColumn(
            target_primary_key,
            coalesce(col("SilverRecordId"), expr("uuid()"))
        ).drop("SilverRecordId")
    
    def _execute_upsert(self, df: DataFrame, target_table: str, target_primary_key: str, merge_sql: Optional[str] = None):
        """
        Run the SQL MERGE command to apply inserts and updates.
        
        You can provide custom MERGE SQL for special cases (e.g., Country table
        with case-insensitive name matching). Otherwise, we auto-generate standard
        MERGE logic based on which columns are available.
        """
        
        df.createOrReplaceTempView("_upsert_source_temp")
        target_full = f"{self.target_lakehouse}.{target_table}"
        
        if merge_sql:            
            final_sql = merge_sql.format(
                TARGET_TABLE=target_full,
                SOURCE_VIEW="_upsert_source_temp",
                TARGET_PK=target_primary_key
            )
            
            logging.debug(f"📋 Final SQL (AFTER substitution):")
            logging.debug(final_sql)
        else:
            # Auto-generate MERGE SQL based on available columns
            target_cols = [f.name for f in spark.read.table(target_full).schema.fields]
            available_cols = [c for c in target_cols if c in df.columns]
            
            # Build UPDATE SET clause - use backticks to preserve case-sensitivity
            update_set = ", ".join([f"`{c}` = s.`{c}`" for c in available_cols if c != target_primary_key])
            
            # Build INSERT clause - use backticks to preserve case-sensitivity
            insert_cols = ", ".join([f"`{c}`" for c in available_cols])
            insert_vals = ", ".join([f"s.`{c}`" for c in available_cols])
            
            final_sql = f"""
            MERGE INTO {target_full} AS t
            USING _upsert_source_temp AS s
              ON t.`{target_primary_key}` = s.`{target_primary_key}`
            WHEN MATCHED THEN UPDATE SET {update_set}
            WHEN NOT MATCHED THEN INSERT ({insert_cols}) VALUES ({insert_vals})
            """
        
        spark.sql(final_sql)
        spark.catalog.dropTempView("_upsert_source_temp")
        logging.info(f"✓ Upserted records into {target_table}")
    
    def _update_mapping(self, df: DataFrame, target_table: str, target_primary_key: str, mapping_table_name: Optional[str] = None):
        """
        Register the source ID → Silver GUID relationships in the mapping table.
        
        This is our "phone book" that lets us find Silver records when we know
        their source system ID. Critical for updates and deletes in future syncs.
        
        Args:
            mapping_table_name: Override SourceTable value. Use when target differs from logical source.
                              If None, uses target_table (default behavior).
        """
        # Use override if specified, otherwise default to target_table
        source_table_value = mapping_table_name or target_table
        
        mapping_df = df.select(
            col(target_primary_key).alias("SilverRecordId"),
            col("SourceId"),
            col("SourceSystemId"),
            lit(source_table_value).alias("SourceTable")
        )
        
        mapping_df.createOrReplaceTempView("_mapping_source_temp")
        
        merge_sql = f"""
        MERGE INTO {self.mapping_table} AS t
        USING _mapping_source_temp AS s
          ON t.SilverRecordId = s.SilverRecordId
        WHEN MATCHED THEN UPDATE SET
            t.SourceId = s.SourceId,
            t.SourceSystemId = s.SourceSystemId,
            t.SourceTable = s.SourceTable
        WHEN NOT MATCHED THEN INSERT (SilverRecordId, SourceId, SourceSystemId, SourceTable)
        VALUES (s.SilverRecordId, s.SourceId, s.SourceSystemId, s.SourceTable)
        """
        
        spark.sql(merge_sql)
        spark.catalog.dropTempView("_mapping_source_temp")

### Dynamics 365 data sync

In [ ]:
from typing import List, Dict, Optional, Callable
from pyspark.sql import DataFrame
from pyspark.sql.functions import col, lit
import logging


class Dynamics365DataSync(Dynamics365BaseSync):
    """
    Main orchestration class for Dynamics 365 Bronze-to-Silver synchronization.
    
    Uses composition pattern with specialized components:
    - DataReader: Reads data from Bronze lakehouse
    - FKResolver: Resolves Bronze FKs to Silver GUIDs  
    - DataWriter: Writes data to Silver lakehouse
    
    Features:
    - Watermark-based incremental synchronization
    - Automatic GUID generation for Silver IDs
    - SourceSystemIdMapping maintenance
    - Hard delete support with optional policy
    - Custom transformation functions
    - Foreign key resolution
    """
    
    def __init__(
        self, 
        source_id: str, 
        source_lakehouse: str, 
        target_lakehouse: str,
        watermark_column: str = "SinkModifiedOn",
        delete_flag_column: str = "IsDelete",
        enable_hard_delete: bool = True
    ):
        """
        Initialize the Dynamics 365 data synchronizer with composed components.
        
        Args:
            source_id: SourceId GUID for this data source
            source_lakehouse: Bronze lakehouse name
            target_lakehouse: Silver lakehouse name
            watermark_column: Default column for incremental sync tracking (default: "SinkModifiedOn")
            delete_flag_column: Default column indicating deleted records (default: "IsDelete")
            enable_hard_delete: Whether to process hard deletes by default (default: True)
        """
        super().__init__(source_id, source_lakehouse, target_lakehouse)
        
        # Initialize watermark service
        self.watermark_service = Dynamics365Watermark(source_lakehouse, target_lakehouse)
        self.watermark_column = watermark_column
        self.delete_flag_column = delete_flag_column
        self.enable_hard_delete = enable_hard_delete
        
        # Composition: Initialize component objects
        self.reader = Dynamics365DataReader(
            source_lakehouse=source_lakehouse,
            watermark_column=watermark_column,
            delete_flag_column=delete_flag_column
        )
        
        self.resolver = Dynamics365ForeignKeyResolver(
            source_id=source_id,
            source_lakehouse=source_lakehouse,
            target_lakehouse=target_lakehouse,
            mapping_table=self.mapping_table
        )
        
        self.writer = Dynamics365DataWriter(
            source_id=source_id,
            target_lakehouse=target_lakehouse,
            mapping_table=self.mapping_table,
            enable_hard_delete=enable_hard_delete
        )
    
    # region Public API - Main Entry Points
    
    def sync_table(
        self,
        source_table: str,
        source_primary_key: str,
        source_columns: List[str],
        target_table: str,
        target_primary_key: str,
        transform_func: Optional[Callable[[DataFrame], DataFrame]] = None,
        delete_policy_func: Optional[Callable[[DataFrame], DataFrame]] = None,
        fk_mappings: Optional[Dict[str, str]] = None,
        merge_sql: Optional[str] = None,
        mapping_source_table: Optional[str] = None,
        source_table_lakehouse: Optional[str] = None
    ):
        """
        Synchronize a single data table from Bronze to Silver.
        
        This is the MAIN ENTRY POINT for standard entity synchronization.
        Orchestrates the full sync flow using composed components.
        
        Args:
            source_table: Bronze table name (e.g., "campaign", "contact")
            source_primary_key: Bronze PK column name (e.g., "Id")
            source_columns: List of columns to read from Bronze
            target_table: Silver table name (e.g., "Campaign", "Contact")
            target_primary_key: Silver PK column name (e.g., "CampaignId", "ContactId")
            transform_func: Optional function to transform Bronze data before sync.
                          Receives DataFrame with Bronze columns PLUS auto-resolved FK columns.
                          Auto-resolved columns are named: {source_column}_SilverRecordId
                          Should return DataFrame with Silver schema.
                          Note: SourceId and SourceSystemId are automatically added if missing.
            delete_policy_func: Optional function to filter deletes before execution.
                              Receives DataFrame with Silver PKs to delete.
                              Should return filtered DataFrame.
            fk_mappings: Optional dict mapping Bronze FK columns to Silver tables.
                        Format: {"bronze_column": "SilverTableName"}
                        Example: {"typecode": "CampaignType", "address1_addressid": "Address"}
                        Auto-creates columns: typecode_SilverRecordId, address1_addressid_SilverRecordId
            merge_sql: Optional custom MERGE SQL statement for fine-grained control.
                      Use placeholders: {TARGET_TABLE}, {SOURCE_VIEW}, {TARGET_PK}
                      If not provided, auto-generates MERGE based on columns.
            mapping_source_table: Override SourceTable value in SourceSystemIdMapping.
                                Use when target table differs from logical source (e.g., Constituent).
                                If None, uses target_table (default behavior).
            source_table_lakehouse: Lakehouse prefix for source_table. If None, uses configured Bronze lakehouse.
                                   Set to empty string "" to read without prefix (for TEMPORARY VIEW).
        """
        logging.info(f"🔄 Starting sync: {source_table} → {target_table}")
        
        # Use override if specified, otherwise default to target_table
        actual_mapping_table = mapping_source_table or target_table
        
        # 1. Determine which records need to be processed (only new/modified since last run)
        last_watermark = self.watermark_service.get_last_watermark(source_table, target_table)
        current_watermark = self.watermark_service.get_current_watermark(source_table, self.watermark_column, source_table_lakehouse)
        
        if self._is_already_synced(last_watermark, current_watermark):
            logging.info(f"✅ {target_table} is up to date (watermark: {last_watermark})")
            return
        
        logging.info(f"📊 Sync window: {last_watermark} → {current_watermark}")
        
        # 2. Load only records that changed since last sync (READER component)
        df_changes = self.reader.read_incremental_changes(
            source_table, source_primary_key, source_columns,
            last_watermark, current_watermark, source_table_lakehouse
        )
        
        if df_changes.isEmpty():
            self._complete_sync_without_changes(source_table, target_table, current_watermark)
            return
        
        # 3. Separate records into two groups: those to delete vs. those to insert/update (READER component)
        df_deletes = self.reader.extract_deletes(df_changes, source_primary_key)
        df_active = self.reader.extract_active(df_changes, source_primary_key)

        # 4. Connect related data - resolve FKs (RESOLVER component)
        if not df_active.isEmpty() and fk_mappings:
            df_active = self.resolver.resolve_foreign_keys(df_active, fk_mappings)
        
        # 5. Apply all changes to the target table (WRITER component orchestration)
        self._execute_sync_operations(
            df_deletes, df_active,
            source_primary_key, target_table, target_primary_key,
            delete_policy_func, transform_func, merge_sql, actual_mapping_table
        )
        
        # 6. Remember where we finished so next sync starts from here
        self._finalize_sync(source_table, target_table, current_watermark)
    
    def sync_activity_table(
        self,
        *,
        source_table: str,
        source_primary_key: str,
        source_columns: list,
        target_table: str,
        target_primary_key: str,
        activity_party_join_func: Callable[[DataFrame], DataFrame],
        transform_func: Optional[Callable[[DataFrame], DataFrame]] = None,
        fk_mappings: Optional[Dict[str, str]] = None,
        merge_sql: Optional[str] = None
    ):
        """
        Synchronize Dynamics 365 activity tables where one Bronze record → N Silver records.
        
        Activities (letter, phonecall, email) explode into multiple Silver records
        based on activityparty relationships. Each activityparty becomes one Silver row.
        
        Pattern:
        1. Bronze activity (letter) → incremental slice
        2. Join with activityparty (1:N explosion)
        3. Resolve FKs (Campaign, Contact/Account → Constituent)
        4. Transform to Silver schema
        5. MERGE with composite key (SourceId, SourceSystemId=ActivityPartyId, ConstituentId)
        
        Args:
            source_table: Bronze activity table name (e.g., "letter")
            source_primary_key: Bronze PK (e.g., "Id")
            source_columns: Columns to read from Bronze
            target_table: Silver table name (e.g., "Letter")
            target_primary_key: Silver PK (e.g., "LetterId")
            activity_party_join_func: Function that joins activity → activityparty + enrichment.
                                     Input: incremental activity df
                                     Output: exploded df with ActivityPartyId column
            transform_func: Optional final transformation to Silver schema
            fk_mappings: Standard FK mappings (e.g., {"regardingobjectid": "Campaign"})
            merge_sql: Custom MERGE SQL with composite key matching
        
        Returns:
            None
        
        Example activity_party_join_func:
            def enrich_letter(df):
                # Join activity → activityparty (1:N explosion)
                # Resolve Campaign (regardingobjectid → Campaign)
                # Resolve Contact/Account → Constituent
                # Return df with ActivityPartyId as SourceSystemId
                return df
        """
        logging.info(f"🔄 Starting activity sync: {source_table} → {target_table}")
        
        # 1. Watermark-based incremental read
        last_watermark = self.watermark_service.get_last_watermark(source_table, target_table)
        current_watermark = self.watermark_service.get_current_watermark(source_table, self.watermark_column, None)
        
        if self._is_already_synced(last_watermark, current_watermark):
            logging.info(f"✅ {target_table} is up to date (watermark: {last_watermark})")
            return
        
        logging.info(f"📊 Sync window: {last_watermark} → {current_watermark}")
        
        # 2. Load incremental changes (READER component)
        df_changes = self.reader.read_incremental_changes(
            source_table, source_primary_key, source_columns,
            last_watermark, current_watermark
        )
        
        if df_changes.isEmpty():
            self._complete_sync_without_changes(source_table, target_table, current_watermark)
            return
        
        # 3. Activity-specific: no delete handling (activities don't typically delete via IsDelete)
        # Activities are event-based - once sent, they exist forever
        
        # 4. Activity party explosion + enrichment
        logging.info(f"🎯 Joining with activityparty (1:N explosion)")
        df_exploded = activity_party_join_func(df_changes)
        
        if df_exploded.isEmpty():
            logging.warning(f"⚠️  No activity parties found for {source_table} in sync window")
            self._complete_sync_without_changes(source_table, target_table, current_watermark)
            return
        
        # 5. Resolve FKs (RESOLVER component)
        if fk_mappings:
            df_exploded = self.resolver.resolve_foreign_keys(df_exploded, fk_mappings)
        
        # 6. Apply transformation
        if transform_func:
            df_exploded = transform_func(df_exploded)
        
        # 7. Add SourceId
        df_exploded = df_exploded.withColumn("SourceId", lit(self.source_id))
        
        # 8. Validate required columns
        if "SourceSystemId" not in df_exploded.columns:
            raise ValueError(f"Activity join func must produce 'SourceSystemId' column (should be ActivityPartyId)")
        
        # 9. Execute MERGE with custom SQL (composite key)
        if not merge_sql:
            raise ValueError(f"Activity sync requires custom merge_sql with composite key (SourceId, SourceSystemId, ConstituentId)")
        
        # Process upserts using WRITER component
        self.writer.process_upserts(
            df_active=df_exploded,
            source_primary_key="SourceSystemId",  # ActivityPartyId
            target_table=target_table,
            target_primary_key=target_primary_key,
            transform_func=None,  # Already transformed
            merge_sql=merge_sql,
            mapping_table_name=target_table
        )
        
        # 10. Update watermark
        self._finalize_sync(source_table, target_table, current_watermark)
    
    # endregion
    
    # region Public API - Convenience Delegates (delegate to components)
    
    def resolve_foreign_keys(self, df: DataFrame, fk_mappings: Dict[str, str]) -> DataFrame:
        """
        Delegate to FK resolver component.
        Public method for custom transformations that need additional FK resolution.
        """
        return self.resolver.resolve_foreign_keys(df, fk_mappings)
    
    def resolve_constituent_id(
        self,
        df: DataFrame,
        contact_fk_column: Optional[str] = None,
        account_fk_column: Optional[str] = None,
        output_column: str = "ConstituentId"
    ) -> DataFrame:
        """
        Delegate to FK resolver component.
        Public method for resolving Constituent from Contact/Account.
        """
        return self.resolver.resolve_constituent_id(df, contact_fk_column, account_fk_column, output_column)
    
    def resolve_polymorphic_lookup(
        self,
        df: DataFrame,
        lookup_id_column: str,
        entity_type_column: str,
        entity_type_mappings: dict
    ) -> DataFrame:
        """
        Delegate to FK resolver component.
        Public method for resolving polymorphic lookups.
        """
        return self.resolver.resolve_polymorphic_lookup(df, lookup_id_column, entity_type_column, entity_type_mappings)
    
    def read_activity_parties(
        self,
        activity_id_column: str = "activityid",
        participation_types: list = [1, 2]
    ) -> DataFrame:
        """
        Delegate to reader component.
        Public method for reading activityparty records.
        """
        return self.reader.read_activity_parties(activity_id_column, participation_types)
    
    def read_bronze_lookup(
        self,
        table_name: str,
        columns: dict,
        filters: Optional[List] = None
    ) -> DataFrame:
        """
        Delegate to reader component.
        Public method for reading Bronze lookup tables.
        """
        return self.reader.read_bronze_lookup(table_name, columns, filters)
    
    # endregion
    
    # region Private Helpers - Watermark Management
    
    def _is_already_synced(self, last_watermark, current_watermark) -> bool:
        """
        Check if we've already processed all available data.
        
        Returns True when the newest record in source is older than what we've already synced.
        This means there's nothing new to process.
        """
        return (current_watermark is not None and 
                last_watermark is not None and 
                current_watermark <= last_watermark)
    
    def _complete_sync_without_changes(self, source_table: str, target_table: str, current_watermark):
        """
        Finish the sync when no data changed since last run.
        
        Even though nothing changed, we still update our bookmark (watermark)
        so we know we checked at this point in time.
        """
        logging.info(f"ℹ️  No changes detected for {target_table}")
        self.watermark_service.update_watermark(source_table, target_table, current_watermark)
    
    def _finalize_sync(self, source_table: str, target_table: str, current_watermark):
        """
        Mark this sync as complete and save our progress.
        
        The watermark acts like a bookmark - it tells the next sync run
        where to start reading, so we don't re-process old data.
        """
        self.watermark_service.update_watermark(source_table, target_table, current_watermark)
        logging.info(f"✅ Sync complete: {target_table}")
    
    # endregion
    
    # region Private Helpers - Sync Orchestration
    
    def _execute_sync_operations(
        self,
        df_deletes: DataFrame,
        df_active: DataFrame,
        source_primary_key: str,
        target_table: str,
        target_primary_key: str,
        delete_policy_func: Optional[Callable[[DataFrame], DataFrame]],
        transform_func: Optional[Callable[[DataFrame], DataFrame]],
        merge_sql: Optional[str],
        mapping_table_name: Optional[str] = None
    ):
        """
        Apply all data changes to the target table in the correct sequence.
        
        We always delete first, then insert/update. This order prevents conflicts
        when a record is deleted and re-created with the same ID.
        
        Delegates actual operations to WRITER component.
        """
        # Remove records that were deleted in the source system (WRITER component)
        self.writer.process_deletes(
            df_deletes, source_primary_key, target_table,
            target_primary_key, delete_policy_func
        )
        
        # Add new records and update existing ones with fresh data (WRITER component)
        self.writer.process_upserts(
            df_active, source_primary_key, target_table,
            target_primary_key, transform_func, merge_sql, mapping_table_name
        )
    
    # endregion

## Helper Functions

Utility functions for common data transformation patterns in Dynamics 365 synchronization.

**D365TransformHelpers Class:**
Static utility class containing all reusable helper functions. All methods are static and can be called directly:
```python
D365TransformHelpers.deduplicate_by_window(df, "country", "id")
D365TransformHelpers.join_case_insensitive(df1, df2, "Name", "CountryName", "left")
```

**Available Methods:**
- `deduplicate_by_window()` - Deduplicate DataFrame using window function with row_number
- `normalize_for_matching()` - Normalize string column for case-insensitive matching (lower/upper + trim)
- `join_case_insensitive()` - Join DataFrames with automatic case-insensitive string matching

**Usage Examples:**
```python
# Deduplicate by window
df_clean = D365TransformHelpers.deduplicate_by_window(
    df, 
    partition_by="customer_id",
    order_by="modified_date"
)

# Case-insensitive join (Country lookup)
result = D365TransformHelpers.join_case_insensitive(
    address_df, country_df,
    left_column="CountryName",
    right_column="Name",
    how="left"
)

# Manual normalization (when join helper doesn't fit)
df = D365TransformHelpers.normalize_for_matching(df, "Name", case_mode="upper")
```


In [ ]:
from pyspark.sql import DataFrame, Window
from pyspark.sql.functions import col, row_number
from typing import Union, List, Optional


class D365TransformHelpers:
    """
    Static utility class containing reusable helper functions for Dynamics 365 transformations.
    
    All methods are static and can be called directly:
        D365TransformHelpers.deduplicate_by_window(df, "country", "id")
    
    This class groups common transformation patterns to:
    - Reduce code duplication
    - Improve maintainability
    - Provide consistent patterns across transformations
    """
    
    @staticmethod
    def deduplicate_by_window(
        df: DataFrame,
        partition_by: Union[str, List[str], any],
        order_by: Union[str, List[str], any],
        keep: str = "first"
    ) -> DataFrame:
        """
        Deduplicate DataFrame using window function with row_number.
        
        This is a common pattern in Dynamics 365 transformations where we need to:
        - Pick one record per group (e.g., one Country per name)
        - Select based on priority ordering (e.g., prefer active records)
        - Remove duplicates after normalization
        
        Args:
            df: Input DataFrame
            partition_by: Column(s) to partition by. Can be:
                         - String column name: "country"
                         - List of strings: ["opportunityid", "statecode"]
                         - Column expression: F.upper(F.col("Name"))
                         - List of Column expressions
            order_by: Column(s) to order by (determines which record to keep). Can be:
                     - String column name: "customeraddressid"
                     - List of strings: ["statecode", "modified"]
                     - Column expression: col("modified").desc()
                     - List of Column expressions (complex ordering)
            keep: Which record to keep - "first" or "last" (default: "first")
                 Note: "first" keeps first record after ordering (row_number = 1)
        
        Returns:
            Deduplicated DataFrame with one record per partition
        
        Examples:
            # Simple case: deduplicate by country name, keep first customeraddressid
            D365TransformHelpers.deduplicate_by_window(df, "country_normalized", "customeraddressid")
            
            # Multiple partition columns
            D365TransformHelpers.deduplicate_by_window(df, ["opportunityid", "type"], "modified")
            
            # Complex ordering with Column expressions
            D365TransformHelpers.deduplicate_by_window(
                df, 
                "opportunityid",
                [when(col("statecode") == 0, 0).otherwise(1).asc(),
                 col("modified").desc_nulls_last()]
            )
            
            # Using Column expressions for partition (normalized name matching)
            D365TransformHelpers.deduplicate_by_window(
                df,
                F.upper(F.trim(F.col("Name"))),
                "CreatedDate"
            )
        
        Technical notes:
            - Uses window function with row_number (efficient for large datasets)
            - Temporary column __rn__ is used internally and cleaned up
            - When keep="last", window ordering is reversed automatically
        """
        # Normalize partition_by to list
        if isinstance(partition_by, str):
            partition_cols = [col(partition_by)]
        elif isinstance(partition_by, list):
            # Check if list contains strings or Column objects
            partition_cols = [col(c) if isinstance(c, str) else c for c in partition_by]
        else:
            # Single Column expression
            partition_cols = [partition_by]
        
        # Normalize order_by to list
        if isinstance(order_by, str):
            order_cols = [col(order_by)]
        elif isinstance(order_by, list):
            # Check if list contains strings or Column objects
            order_cols = [col(c) if isinstance(c, str) else c for c in order_by]
        else:
            # Single Column expression
            order_cols = [order_by]
        
        # Reverse ordering if keep="last"
        if keep == "last":
            # Reverse each order column (asc → desc, desc → asc)
            reversed_cols = []
            for order_col in order_cols:
                # This is a simplified approach - for full implementation would need to detect
                # current sort direction and reverse it
                reversed_cols.append(order_col.desc())
            order_cols = reversed_cols
        
        # Build window specification
        w = Window.partitionBy(*partition_cols).orderBy(*order_cols)
        
        # Apply row_number, filter, and cleanup
        return (df.withColumn("__rn__", row_number().over(w))
                 .filter(col("__rn__") == 1)
                 .drop("__rn__"))
    
    @staticmethod
    def normalize_for_matching(
        df: DataFrame,
        column: str,
        output_column: Optional[str] = None,
        case_mode: str = "lower"
    ) -> DataFrame:
        """
        Normalize string column for case-insensitive matching.
        
        Common pattern in Dynamics 365 where entity names, lookup values, and text fields
        need case-insensitive comparison. Applies trim() + upper()/lower() normalization.
        
        Args:
            df: Input DataFrame
            column: Column name to normalize
            output_column: Name for normalized column (default: "__normalized_{column}")
            case_mode: "lower" or "upper" normalization (default: "lower")
        
        Returns:
            DataFrame with additional normalized column
        
        Examples:
            # Add normalized Name column for matching
            df = D365TransformHelpers.normalize_for_matching(df, "Name")
            # Result: adds column __normalized_Name with lower(trim(Name))
            
            # Custom output column name
            df = D365TransformHelpers.normalize_for_matching(
                df, "CountryName", 
                output_column="country_normalized",
                case_mode="upper"
            )
            
            # Multiple normalizations for complex matching
            df = D365TransformHelpers.normalize_for_matching(df, "FirstName")
            df = D365TransformHelpers.normalize_for_matching(df, "LastName")
        
        Technical notes:
            - Uses lower(trim()) by default for case-insensitive matching
            - Temporary normalized columns should be dropped after join/comparison
            - Prefer using join_case_insensitive() for direct joins
        """
        from pyspark.sql.functions import upper, lower, trim
        
        if output_column is None:
            output_column = f"__normalized_{column}"
        
        case_func = upper if case_mode == "upper" else lower
        
        return df.withColumn(
            output_column,
            case_func(trim(col(column)))
        )
    
    @staticmethod
    def join_case_insensitive(
        left_df: DataFrame,
        right_df: DataFrame,
        left_column: str,
        right_column: str,
        how: str = "inner"
    ) -> DataFrame:
        """
        Join DataFrames with case-insensitive string matching.
        
        Common pattern in Dynamics 365 lookups where entity names or text fields
        need to be matched regardless of case. Automatically normalizes join columns
        and cleans up temporary columns.
        
        Args:
            left_df: Left DataFrame
            right_df: Right DataFrame
            left_column: Column name in left DataFrame to join on
            right_column: Column name in right DataFrame to join on
            how: Join type - "inner", "left", "right", "outer" (default: "inner")
        
        Returns:
            Joined DataFrame with temporary normalized columns removed
        
        Examples:
            # Country lookup by name (case-insensitive)
            result = D365TransformHelpers.join_case_insensitive(
                address_df, country_df,
                left_column="CountryName",
                right_column="Name",
                how="left"
            )
            
            # ConstituentType matching
            result = D365TransformHelpers.join_case_insensitive(
                source_df, constituent_type_df,
                left_column="TypeName",
                right_column="Name",
                how="inner"
            )
            
            # Multiple case-insensitive joins in sequence
            df = D365TransformHelpers.join_case_insensitive(df, lookup1, "Field1", "Name")
            df = D365TransformHelpers.join_case_insensitive(df, lookup2, "Field2", "Code")
        
        Technical notes:
            - Uses lower(trim()) for normalization
            - Automatically cleans up __left_norm__ and __right_norm__ columns
            - More efficient than manual normalize + join + cleanup pattern
        """
        from pyspark.sql.functions import lower, trim
        
        # Normalize both sides
        left_norm = left_df.withColumn("__left_norm__", lower(trim(col(left_column))))
        right_norm = right_df.withColumn("__right_norm__", lower(trim(col(right_column))))
        
        # Join on normalized columns
        result = left_norm.join(
            right_norm,
            col("__left_norm__") == col("__right_norm__"),
            how=how
        )
        
        # Cleanup temporary columns
        return result.drop("__left_norm__", "__right_norm__")